In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [11]:
pd.read_csv('E:/hms-harmful-brain-activity-classification/train.csv')
train = pd.read_csv('E:/hms-harmful-brain-activity-classification/train.csv')
train.shape
train.head()

,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,1628180742,0,0.0,353733,0,0.0,127492639,42516,Seizure,3,0,0,0,0,0
1,1628180742,1,6.0,353733,1,6.0,3887563113,42516,Seizure,3,0,0,0,0,0
2,1628180742,2,8.0,353733,2,8.0,1142670488,42516,Seizure,3,0,0,0,0,0
3,1628180742,3,18.0,353733,3,18.0,2718991173,42516,Seizure,3,0,0,0,0,0
4,1628180742,4,24.0,353733,4,24.0,3080632009,42516,Seizure,3,0,0,0,0,0


## Settings 

In [ ]:
CREATE_EEG = False
TRAIN_MODEL = True


## RAW_EEG_FEATURES

In [14]:
df = pd.read_parquet('E:/hms-harmful-brain-activity-classification/train_eegs/1000913311.parquet')
FEATS = df.columns
print(f'There are {len(FEATS)} raw eeg features')
print(list(FEATS))

There are 20 raw eeg features
['Fp1', 'F3', 'C3', 'P3', 'F7', 'T3', 'T5', 'O1', 'Fz', 'Cz', 'Pz', 'Fp2', 'F4', 'C4', 'P4', 'F8', 'T4', 'T6', 'O2', 'EKG']


In [18]:
type(FEATS)
FEATS = ['Fp1','T3','C3','O1','C4','T4','O2','Fp2']
FEATS2IDX = {x:y for x,y in zip(FEATS, range(len(FEATS)))}
print (list(FEATS))
for x,y in zip(FEATS,range(len(FEATS))):
    print(x)
    print(y)

['Fp1', 'T3', 'C3', 'O1', 'C4', 'T4', 'O2', 'Fp2']
Fp1
0
T3
1
C3
2
O1
3
C4
4
T4
5
O2
6
Fp2
7
